## Calculating Clusters
This is a very simple tutorial demonstrating how to calculate clusters from a MDTraj trajectory.  We will be using the simple example from the paper: doi 10.1063/1.454720, where the clustering algorithm was derived.

## Installation
This package only depends on `NumPy` and `MDTraj`.  These can either be installed by running `pip install -r requirements.txt` at the root directory of this package, or `conda install --file requirements.txt`.

Once the dependencies have been installed, `pairing` can be installed by running `pip install -e .`

In [ ]:
import pairing
import numpy as np

## Checklist before starting
- Ensure that the molecules in your trajectory are properly wrapped inside the simulation box.  That is, make sure that molecules are not across the PBC.  With GROMACS, this can be done by running `gmx trjconv` on the command line.
- Determine the distance criteria for your system.  This will determine which molecules are paired in your system.

## Direct Correlation Matrix
The first step is to calculate the direct correlation matrix of your system.  This is a sparse matrix in which position [i,i] in the matrix represents molecule i and position [i,j] represents the pairing of molecule i and molecule j.  A value of 1 represents two molecules are paired, and a value of 0 represents two molecules are unpaired.

The function to calculate this is `pairing.calc_direct`.  This function has two arguments: `trj` and `cutoff`.  `trj` is a MDTraj trajectory, and `cutoff` is the distance cutoff that determines whether or not two molecules are paired.

For this tutorial, we will not be calling this function.  Instead, we will be using the example presented in doi 10.1063/1.454720.  Please refer to Fig. 12 which shows a system of 5 beads.  In this system, bead 1 is paired to bead 5, bead 5 is paired to bead 3, bead 3 is paired to bead 2, and bead 4 is unpaired.  As a result, the following direct correlation matrix represents this system:

```
direct = np.asarray([[[1, 0, 0, 0, 1],
                      [0, 1, 1, 0, 0],                 
                      [0, 1, 1, 0, 1],
                      [0, 0, 0, 1, 0],
                      [1, 0, 1, 0, 1]]], dtype=np.int32)
```                  

Let's take a look at the first row, which represents bead 1.  The first index (0) has a value of 1, since this index represents bead 1 itself.  Indices 1, 2, and 3 have values of 0, since bead 1 in not paired to beads 2, 3, or 4.  Finally, index 4 has a value of 1, since beads 1 and 5 are paired.

## Indirect Correlation Matrix
The next step is to calculate the indirect correlation matrix, which represents the molecules that are indirectly paired in a given system.  Sticking with the same example, beads 1,2,3, and 5 are defined as indirectly connected to eachother.  The indirectly paired molecules are how we will count the clusters in our system.

To calculate this, the `pairing.calc_indirect` function will be called.  The argument for this function is the direct correlation matrix.

In [ ]:
direct = np.asarray([[[1, 0, 0, 0, 1],
                      [0, 1, 1, 0, 0],                 
                      [0, 1, 1, 0, 1],
                      [0, 0, 0, 1, 0],
                      [1, 0, 1, 0, 1]]], dtype=np.int32)

indirect = pairing.calc_indirect(direct)
print(indirect)

The resulting indirect matrix should match A3 in doi 10.1063/1.454720.  Looking at row 1, we can see that beads 1, 2, 3, and 5 are indirectly paired.  Further, the fourth row shows that bead 4 is unpaired.

## Reducing the indirect correlation matrix
Now that that indirect correlation matrix has been calculated, we can now calculate the number of clusters in our system.  If you look at the indirect matrix, you may notice that the rows that represent beads 1, 2, 3, and 5 are identical.  Based on this, we can reduce the matrix so that we have a matrix of our clusters in the system.

To do this, we will use the function `pairing.calc_reduc`.  The argument for this function is the indirect correlation matrix.

In [ ]:
reduced = pairing.calc_reduc(indirect)
print(reduced)

Looking at `reduced`, we now have a matrix with two columns.  The first column represents the first cluster in this system, which is the unpaired bead 4.  The second column represents the second cluster in our system, which includes beads 1, 2, 3, and 5.  With this information, we can calculate a) How many clusters are in the system, and b) how large each cluster is.